In [15]:
import pandas as pd
from perp_bitget import PerpBitget
import ccxt
import numpy as np
import requests
import json
import time
import os
from datetime import datetime

now = datetime.now()
current_time = now.strftime("%d/%m/%Y %H:%M:%S")
print("--- Start Execution Time :", current_time, "---")



# Connection to API (To update in the AWS' server)
f = open(
    "../api_connection.json")
api_connection = json.load(f)
f.close()

account_to_select = "bitget_connection"
production = True

pair = "BTC/USDT:USDT"
timeframe = "1d"
leverage_long = 2
leverage_short = 1

print(f"--- {pair} timeframe: {timeframe} Leverage Long:  x {leverage_long} ---")
print(f"--- {pair} timeframe:{timeframe} Leverage Short: x {leverage_short} ---")


type = ["long", "short"]


--- Start Execution Time : 18/04/2023 17:25:24 ---
--- BTC/USDT:USDT timeframe: 1d Leverage Long:  x 2 ---
--- BTC/USDT:USDT timeframe:1d Leverage Short: x 1 ---


In [16]:
def btc():
    url = "https://www.bitstamp.net/api/v2/ohlc/btcusd/"
    timeframe = 86400
    start_date = 1312174800
    end_date = int(time.time())

    # set the maximum number of data points to return per request
    limit = 1000

    data_list = []
    while start_date <= end_date:

        request_end_date = min(start_date + limit*timeframe, end_date)
        response = requests.get(url + "?step=" + str(timeframe) + "&start=" + str(start_date) + "&end=" + str(request_end_date) + "&limit=" + str(limit))
        data = json.loads(response.text)
        data_list += data['data']['ohlc']
        start_date += limit*timeframe

    df = pd.DataFrame(data_list)
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
    df[['open', 'high', 'low', 'close', 'volume']] = df[['open', 'high', 'low', 'close', 'volume']].astype(float)

    df.set_index('timestamp',drop=True, inplace = True)
    #df.columns = [e.capitalize() for e in df.columns]
    df.sort_index(ascending = True, inplace= True)

    df.drop_duplicates(inplace = True)

    period_1 = 30
    period_2 = 120

    sqrt_period_30 = np.sqrt(period_1)
    sqrt_period_120 = np.sqrt(period_2)

    def John(x, y):
        alpha = 2 / (y + 1)
        sum = np.zeros(len(x))
        sum[0] = alpha * x[0]
        for i in range(1, len(x)):
            sum[i] = alpha * x[i] + (1 - alpha) * sum[i-1]
        return sum

    close_ema1_30 = John(df['close'], int(period_1 / 2))
    close_ema2_30 = John(df['close'], period_1)
    ehma_30 = John(2 * close_ema1_30 - close_ema2_30, sqrt_period_30)

    df['ehma_30'] = ehma_30
    df['ehma_30_1'] = df['ehma_30'].shift(1)


    close_ema1_120 = John(df['close'], int(period_2 / 2))
    close_ema2_120 = John(df['close'], period_2)
    ehma_120 = John(2 * close_ema1_120 - close_ema2_120, sqrt_period_120)

    df['ehma_120'] = ehma_120
    df['ehma_120_1'] = df['ehma_120'].shift(1)

    return df

In [21]:

# Trading actions
def open_long(row):

    if row['ehma_30'] > row['ehma_30_1'] and row['ehma_120'] > row['ehma_120_1']:
        return True
    else:
        return False


def close_long(row):
    if row['ehma_30'] < row['ehma_30_1'] or  row['ehma_120'] < row['ehma_120_1']:
        return True
    else:
        return False

def open_short(row):
    if row['ehma_30'] < row['ehma_30_1'] and row['ehma_120'] < row['ehma_120_1']:
        return True
    else:
        return False


def close_short(row):
    if row['ehma_30'] > row['ehma_30_1'] or row['ehma_120'] > row['ehma_120_1']:
        return True
    else:
        return False


In [22]:
df = btc().copy()
row = df.iloc[-1]


In [23]:
row

close         30158.000000
high          30483.000000
low           29122.000000
open          29452.000000
volume         1271.261542
ehma_30       30227.626913
ehma_30_1     30119.151887
ehma_120      28007.751215
ehma_120_1    27852.432770
Name: 2023-04-18 00:00:00, dtype: float64

In [24]:
open_long(row)

True